In [3]:
using Gurobi
using QPSReader
using QuadraticModels
using NLPModels
using SolverTools
using SolverBenchmark
using DataFrames

┌ Info: Precompiling Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]
└ @ Base loading.jl:1260


In [4]:
show_statuses()

STATUSES:
  :acceptable     => solved to within acceptable tolerances
  :exception      => unhandled exception
  :first_order    => first-order stationary
  :infeasible     => problem may be infeasible
  :max_eval       => maximum number of function evaluations
  :max_iter       => maximum iteration
  :max_time       => maximum elapsed time
  :neg_pred       => negative predicted reduction
  :not_desc       => not a descent direction
  :small_residual => small residual
  :small_step     => step too small
  :stalled        => stalled
  :unbounded      => objective function may be unbounded from below
  :unknown        => unknown
  :user           => user-requested stop


In [5]:
STATUSES

UndefVarError: UndefVarError: STATUSES not defined

In [9]:

function createQuadraticModel(qpdata)
    # probleme du point initial
    #x0 = init_x0(qpdata.lvar, qpdata.uvar)
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0)

end

function display_results_Gurobi(gurobi_model)
    # show results
    sol = get_solution(gurobi_model)
    println("soln = $(sol)")

    objv = get_objval(gurobi_model)
    println("objv = $(objv)")
end
    

function OptimizeGurobi(qpdata)
    QM = createQuadraticModel(qpdata)
    SM = SlackModel(QM)
    
    env = Gurobi.Env()

    # set presolve to 0
    setparam!(env, "Presolve", 0)
    
    model = gurobi_model(env; name = "lp_01", f = grad(SM, zeros(length(SM.meta.x0))),
                Aeq = jac(SM, SM.meta.x0), beq = SM.meta.lcon, 
                lb = SM.meta.lvar, ub = SM.meta.uvar)
     # run optimization
    optimize(model)
    println(env)
    # outputs
    optim_info = get_optiminfo(model)
    if optim_info.status == :optimal
        ExStats_status = :acceptable
    end
    GenExStats = GenericExecutionStats(ExStats_status, SM, solution = get_solution(model),
                               objective = get_objval(model), iter = optim_info.iter_count,
                                elapsed_time = optim_info.runtime)
    println(GenExStats)
    return model
end



OptimizeGurobi (generic function with 1 method)

In [7]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib"
afiro = string(path_pb, "\\AFIRO.SIF")
qpdata2 = readqps(afiro);

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [93]:
model = OptimizeGurobi(qpdata2);


#display_results_Gurobi(model)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 27 rows, 51 columns and 102 nonzeros
Model fingerprint: 0x84b9128a
Coefficient statistics:
  Matrix range     [1e-01, 2e+00]
  Objective range  [3e-01, 1e+01]
  Bounds range     [8e+01, 5e+02]
  RHS range        [4e+01, 4e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.8000000e+30   6.800000e+30   1.800000e+00      0s
      10   -4.6475314e+02   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.00 seconds
Optimal objective -4.647531429e+02
Gurobi.Env(Ptr{Nothing} @0x000000005c30f020)


In [47]:
SM1 = SlackModel(createQuadraticModel(qpdata2))
GenExStats = GenericExecutionStats(:acceptable, SM1, solution = get_solution(model),
                               objective = get_objval(model))

"Execution stats: solved to within acceptable tolerances"

# Test sur tous les pb

In [11]:
i = 0
for file_name in readdir(path_pb)
    if file_name[end-3:end] == ".SIF" && !(file_name in ["BLEND.SIF"; "DFL001.SIF";  #QPSReader not working
                                                        "FORPLAN.SIF"; "GFRD-PNC.SIF"; 
                                                        "SIERRA.SIF"]) #&&
                                        #!(file_name in ["25FV47.SIF" ; "80BAU3B.SIF"])
        
        i += 1
        println(file_name)
        pb_i = string(path_pb, "\\", file_name)
        qpdata_i = readqps(pb_i) 
        model_i = OptimizeGurobi(qpdata_i);

    end
    if i == 2
        break
        break
    end
end

25FV47.SIF


┌ Info: Using '25FV47' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'R0000' as objective (l. 21)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '.00001' as RHS (l. 6793)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 821 rows, 1876 columns and 10705 nonzeros
Model fingerprint: 0x5534e7d0
Coefficient statistics:
  Matrix range     [2e-04, 2e+02]
  Objective range  [1e-03, 1e+02]
  Bounds range     [1e+00, 2e+03]
  RHS range        [2e-01, 6e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.2233140e+32   1.045097e+32   1.223314e+02      0s
    2674    5.5018459e+03   0.000000e+00   0.000000e+00      0s

Solved in 2674 iterations and 0.35 seconds
Optimal objective  5.501845888e+03
Gurobi.Env(Ptr{Nothing} @0x000000004bc76fb0)
Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 5501.845888286749
  primal feasibility: Inf
  dual feasibility: Inf
  solution: [53.13886510118529  0.0  0.0  34.22576854795369 ⋯ 247.0]
  iterations: 2674
  elapsed time: 0.3548583984375
80BAU3B.SIF


┌ Info: Using '80BAU3B' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'HOLLY' as objective (l. 375)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS1' as RHS (l. 19765)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'BOUNDROW' as BOUNDS (l. 19939)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622


Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2262 rows, 12061 columns and 23264 nonzeros
Model fingerprint: 0x10eb89b7
Coefficient statistics:
  Matrix range     [2e-04, 1e+02]
  Objective range  [1e-03, 3e+03]
  Bounds range     [7e-05, 1e+06]
  RHS range        [0e+00, 0e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.1449760e+33   1.384250e+32   1.144976e+03      0s
    5032    9.8722419e+05   0.000000e+00   0.000000e+00      0s

Solved in 5032 iterations and 0.29 seconds
Optimal objective  9.872241924e+05
Gurobi.Env(Ptr{Nothing} @0x000000004bbe9550)
Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 987224.19240909
  primal feasibility: Inf
  dual feasibility: Inf
  solution: [531.1245039018952  286.801320754717  544.7416022099447  895.5392535675082 ⋯ -0.0]
  iterations: 5032
  elapsed time: 0.2852306365966797


In [105]:
for file_name in readdir(path_pb)
    println("problem ", file_name)
    
end

problem 25FV47.SIF
problem 80BAU3B.SIF
problem ADLITTLE.SIF
problem AFIRO.SIF
problem AGG.SIF
problem AGG2.SIF
problem AGG3.SIF
problem BANDM.SIF
problem BEACONFD.SIF
problem BLEND.SIF
problem BNL1.SIF
problem BNL2.SIF
problem BOEING1.SIF
problem BOEING2.SIF
problem BORE3D.SIF
problem BRANDY.SIF
problem CAPRI.SIF
problem CRE-A.SIF
problem CRE-B.SIF
problem CRE-C.SIF
problem CRE-D.SIF
problem CYCLE.SIF
problem CZPROB.SIF
problem D2Q06C.SIF
problem D6CUBE.SIF
problem DEGEN2.SIF
problem DEGEN3.SIF
problem DFL001.SIF
problem E226.SIF
problem ETAMACRO.SIF
problem FFFFF800.SIF
problem FINNIS.SIF
problem FIT1D.SIF
problem FIT1P.SIF
problem FIT2D.SIF
problem FIT2P.SIF
problem FORPLAN.SIF
problem GANGES.SIF
problem GFRD-PNC.SIF
problem GREENBEA.SIF
problem GREENBEB.SIF
problem GROW15.SIF
problem GROW22.SIF
problem GROW7.SIF
problem ISRAEL.SIF
problem KB2.SIF
problem KEN-07.SIF
problem KEN-11.SIF
problem KEN-13.SIF
problem KEN-18.SIF
problem LOTFI.SIF
problem MAROS-R7.SIF
problem MAROS.SIF
probl

In [98]:
T = typeof(model)
for (name, typ) in zip(fieldnames(T), T.types)
    println("type of the fieldname $name is $typ")
end

type of the fieldname env is Gurobi.Env
type of the fieldname ptr_model is Ptr{Nothing}
type of the fieldname callback is Any
type of the fieldname finalize_env is Bool
type of the fieldname conflict is Integer
